In [1]:
import os
import torch
import numpy as np
from tqdm import tqdm

from torch.optim import Adam
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.data.dataloader import DataLoader

from preprocessor import AgeRecognitionPreprocessor
from dataset import AgeRecognitionDataset
from models import vit_l_16_age_recognizer, vit_b_16_age_recognizer, rese
from loss import AgeRecognitionLoss

In [2]:
IMAGE_DIR = './Cleaned/'
TEST_PAIRINGS = './test_data.csv'
BATCH_SIZE = 12
EPOCHES = 1000
DEVICE = 'cuda'

In [3]:
model = vit_b_16_age_recognizer().to(DEVICE)
loss_function = AgeRecognitionLoss().to(DEVICE)
preprocessor = AgeRecognitionPreprocessor()
dataset = AgeRecognitionDataset(triplet_csv_path=TEST_PAIRINGS, image_dir=IMAGE_DIR, preprocessor=preprocessor, kfolds=1, device=DEVICE)

In [4]:
if os.path.exists('./Checkpoint/best.pt'):
    print("Loading best ver...")
    best_state = torch.load('./Checkpoint/best.pt')
    model.load_state_dict(best_state['model_state_dict'])
    loss_function.load_state_dict(best_state['loss_state_dict'])

Loading best ver...


In [6]:
torch.manual_seed(3407)
record = []
model.eval()    
total_loss = 0
for item in tqdm(dataset):
    # Batch shape: (N, Anchor-Positive-Negative, C, H, W)
    predictions = model.forward_features(item)
    testing_loss = loss_function(predictions.unsqueeze(0))
    total_loss += testing_loss.cpu().detach()
print(total_loss / len(dataset))


100%|██████████| 49828/49828 [17:20<00:00, 47.88it/s]

tensor(0.4296)
